In [1]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Move to the parent directory
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

# Print the updated working directory
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)

Current Directory: /mnt/c/users/rjmay/code/research_ms/alt_distr
Updated Directory: /mnt/c/users/rjmay/code/research_ms


In [2]:
! source ../venv/bin/activate

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import pickle
import time



In [67]:
PICKLED =True

dim_str = "15" # TODO change me 


In [68]:
# BER_values = {'beta_beta': 0.10937546875016925, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
#                'uniform_normal': 0.15350691875939732}

BER_values = {'gm': 0.13868461929442688, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
               'uniform_normal': 0.15350691875939732}

In [69]:
for k in BER_values:
    print((k))

gm
paired_uniforms
normal_beta
uniform_normal


In [70]:
wanted_columns = ['Dp', 'Bha',
       'Bha_knn', 'inf', 'enDive']

### Simulation stage

In [71]:
# from modules.multi_bounds_v3 import bounds_class
# # from modules.Bhatt_knn_func import knn_num_calc
# from modules.data_gen import data_gen

In [72]:
# file_path = 'sim_data/beta_beta' + dim_str +'.pkl' # DONT FORGET TO CHANGE ME IF YOU COPY AND PASTE


    

# if PICKLED == True:
    

#     # Open the file in binary read mode
#     with open(file_path, 'rb') as file:
#         # Use pickle.load to deserialize and load the list of objects from the file
#         loaded_objects = pickle.load(file)

#     print('Loaded object successfully')
    
#     bound_obj_lst = loaded_objects

In [73]:
all_obj = {}

for k in BER_values:
    file_path = 'sim_data/' + k + dim_str + '.pkl'
    print(file_path, end = '\t')
    #Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load to deserialize and load the list of objects from the file
        loaded_objects = pickle.load(file)

    print('Loaded object successfully')
    
    all_obj[k] = loaded_objects



sim_data/gm15.pkl	Loaded object successfully
sim_data/paired_uniforms15.pkl	Loaded object successfully
sim_data/normal_beta15.pkl	Loaded object successfully
sim_data/uniform_normal15.pkl	Loaded object successfully


In [74]:
# all_obj

In [75]:
all_obj['gm']

[[<modules.multi_bounds_v3.bounds_class at 0x7fdde36bd180>,
 array([ 100,  145,  211,  307,  447,  650,  945, 1375, 2000])]

### Using the data



In [76]:
sample_sizes = np.logspace(2, 3.3011, 9 , endpoint = True, dtype = int)

dpi = 250

In [77]:
validity_dict = {}

for k in all_obj:
    if len(k) == 2:
        temp_bound_obj_lst =all_obj[k][0]
    else:
        temp_bound_obj_lst = all_obj[k]

    lst = [obj.validity(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns]

    short_df = pd.concat([good_columns.head(1), good_columns.tail(1)])
    
    validity_dict[k] = short_df



In [78]:
validity_dict

{'gm':           Dp  Bha  Bha_knn  inf  enDive
 100   0.0525  0.0    0.235  0.0  0.0100
 2000  0.0250  0.0    0.995  0.0  0.0675,
 'paired_uniforms':        Dp     Bha  Bha_knn  inf  enDive
 100   0.0  0.9875     0.43  0.0  0.0350
 2000  0.0  1.0000     1.00  0.0  0.1975,
 'normal_beta':        Dp     Bha  Bha_knn  inf  enDive
 100   0.0  0.9675   0.7875  0.0  0.0425
 2000  0.0  1.0000   0.0750  0.0  0.1775,
 'uniform_normal':          Dp     Bha  Bha_knn  inf  enDive
 100   0.410  0.6225   0.0325  0.0  0.0700
 2000  0.945  1.0000   0.9900  0.0  0.2475}

In [79]:
# final_df = pd.concat(
#     [df.assign(Key=key) for key, df in validity_dict.items()],
#     ignore_index=True
# ) 

# final_df

In [80]:
#  Assuming validity_dict is populated as per your previous code
final_df_list = []

for key, df in validity_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_validity = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
final_validity

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.0525,0.0000,0.2350,0.0,0.0100,gm_100
1,0.0250,0.0000,0.9950,0.0,0.0675,gm_2000
2,0.0000,0.9875,0.4300,0.0,0.0350,paired_uniforms_100
3,0.0000,1.0000,1.0000,0.0,0.1975,paired_uniforms_2000
4,0.0000,0.9675,0.7875,0.0,0.0425,normal_beta_100
5,0.0000,1.0000,0.0750,0.0,0.1775,normal_beta_2000
6,0.4100,0.6225,0.0325,0.0,0.0700,uniform_normal_100
7,0.9450,1.0000,0.9900,0.0,0.2475,uniform_normal_2000


In [81]:
# lst = [obj.bound_width() for obj in bound_obj_lst ]

# bound_width_df = pd.DataFrame(lst, index = sample_sizes)

# bound_width_df.head()

In [82]:
width_dict = {}

for k in all_obj:
    temp_bound_obj_lst = all_obj[k]
    if len(temp_bound_obj_lst) == 2:
        temp_bound_obj_lst = temp_bound_obj_lst[0]
    

    lst = [obj.bound_width() for obj in temp_bound_obj_lst ]
    temp_df = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = temp_df[wanted_columns]

    short_df = pd.concat([good_columns.head(1), good_columns.tail(1)])
    
    width_dict[k] = short_df

In [83]:
#  Assuming validity_dict is populated as per your previous code
final_df_list2 = []

for key, df in width_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list2.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_width = pd.concat(final_df_list2, ignore_index=True)

# Display the final DataFrame
final_width

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.114117,0.091127,0.092371,0.000011,0.003497,gm_100
1,0.105041,0.198331,0.165250,0.000384,0.016896,gm_2000
2,0.120921,0.168586,0.087037,0.007195,0.021459,paired_uniforms_100
3,0.109410,0.154520,0.173543,0.004645,0.037229,paired_uniforms_2000
4,0.121693,0.172983,0.104246,0.006308,0.021748,normal_beta_100
5,0.120914,0.150620,0.195761,0.005510,0.073859,normal_beta_2000
6,0.106934,0.203268,0.072940,0.006145,0.024226,uniform_normal_100
7,0.102490,0.199001,0.159839,0.005783,0.051378,uniform_normal_2000


In [84]:
final_validity_renamed = final_validity.rename(columns={
    'Dp': 'Dp_valid',
    'Bha': 'Bha_valid',
    'Bha_knn': 'Bha_knn_valid',
    'inf': 'inf_valid',
    'enDive': 'enDive_valid'
})

final_width_renamed = final_width.rename(columns={
    'Dp': 'Dp_width',
    'Bha': 'Bha_width',
    'Bha_knn': 'Bha_knn_width',
    'inf': 'inf_width',
    'enDive': 'enDive_width'
})

# Step 2: Merge the two DataFrames on the 'Key' column
combined_df = pd.merge(final_validity_renamed, final_width_renamed, on='Key')

# Display the combined DataFrame
columns = combined_df.columns.to_list()

columns.sort()
# combined_df.columns
columns

['Bha_knn_valid',
 'Bha_knn_width',
 'Bha_valid',
 'Bha_width',
 'Dp_valid',
 'Dp_width',
 'Key',
 'enDive_valid',
 'enDive_width',
 'inf_valid',
 'inf_width']

In [85]:
order_columns = [ 'Key',
'Bha_valid',
'Bha_width',
'Bha_knn_valid',
'Bha_knn_width',
'inf_valid',
'inf_width',
 'Dp_valid',
 'Dp_width',
 'enDive_valid',
 'enDive_width'
 ]

In [86]:
combined_sorted = combined_df[order_columns]

combined_sorted.index = combined_sorted.Key

combined_sorted= combined_sorted.drop(columns=['Key'])

combined_sorted

,Bha_valid,Bha_width,Bha_knn_valid,Bha_knn_width,inf_valid,inf_width,Dp_valid,Dp_width,enDive_valid,enDive_width
Key,,,,,,,,,,
gm_100,0.0000,0.091127,0.2350,0.092371,0.0,0.000011,0.0525,0.114117,0.0100,0.003497
gm_2000,0.0000,0.198331,0.9950,0.165250,0.0,0.000384,0.0250,0.105041,0.0675,0.016896
paired_uniforms_100,0.9875,0.168586,0.4300,0.087037,0.0,0.007195,0.0000,0.120921,0.0350,0.021459
paired_uniforms_2000,1.0000,0.154520,1.0000,0.173543,0.0,0.004645,0.0000,0.109410,0.1975,0.037229
normal_beta_100,0.9675,0.172983,0.7875,0.104246,0.0,0.006308,0.0000,0.121693,0.0425,0.021748
normal_beta_2000,1.0000,0.150620,0.0750,0.195761,0.0,0.005510,0.0000,0.120914,0.1775,0.073859
uniform_normal_100,0.6225,0.203268,0.0325,0.072940,0.0,0.006145,0.4100,0.106934,0.0700,0.024226
uniform_normal_2000,1.0000,0.199001,0.9900,0.159839,0.0,0.005783,0.9450,0.102490,0.2475,0.051378
